In [1]:
import random
import time

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Random Search

1. Given a target sequence of characters (e.g. `'hello world'`)

2. Create a set of characters, e.g. `['h','e','l','l','o',' ','w','o','r','l','d']`

3. Until target is found...

    a. Produce a guess by randomly combining characters from character set to form a sequence

In [2]:
def random_search(trgt):
    """
    Randomly combine a set of characters to create a sequence the length of the
    target sequence.
    
    Args:
        trgt(str): the target string to be found by search
        
    Returns:
        float: length of runtime of search in seconds
    """
    target, letters = list(trgt), tuple(trgt)
    guess, count = None, 0
    print('TARGET:\t\t{}'.format(trgt))
    
    start = time.time()
    while guess != target:
        guess = random.sample(population=letters, k=len(target))
        if not count%5000:
            print('\rGUESS:\t\t{}'.format(''.join(guess)), end='', flush=True)
        count += 1
    end = time.time()
    
    print('\rOUTPUT:\t\t{}'.format(''.join(guess)), end='')
    print('\nNUM ATTEMPTS:\t{:,}\nTIME:\t\t{:.2f} seconds'.format(count+1, end-start))
    
    return end-start

## Genetic Algorithm

1. Randomly generate initial population

2. Until target is found in population...

    a. Determine fitness of population

    b. Reproduce to generate new population

    c. Mutate new population

    d. Replace old population with new population 

In [3]:
def fitness(pop, target):
    """
    Calculate fitness of each member of population.
    """
    # dictionary in which to store fitness scores
    results = {}
    
    # for each member of the population
    for member in pop:
        score = 0
        # determine the number of character matches (character order matters)
        for index, val in enumerate(member):
            if val == target[index]:
                score += 1
        results[tuple(member)] = score/len(member)
    
    return results


def crossOver(N, chars, prob_list, mutation_rate):
    """
    Mate parents to produce child and apply mutation.
    """
    new_population = []
    for _ in range(N):
        
        # randomly choose two parents
        ind_1, ind_2 = np.random.choice(len(prob_list), size=2, replace=False)
        parent_1, parent_2 = prob_list[ind_1], prob_list[ind_2]
        
        # take parts from each parent
        midpoint = np.random.randint(1, len(parent_1))
        child = list(parent_1)[:midpoint]
        child.extend(list(parent_2)[midpoint:])
        
        # mutate child
        for i in range(len(child)):
            if np.random.random(size=1) <= mutation_rate:
                child[i] = chr(random.choice(chars))
        new_population.append(child)
    
    return new_population


def genetic_algorithm(target, N=200, mutation_rate=0.01):
    """
    Apply steps of genetic algorithm until target is found.
    
    Args:
        N(int): population size
        mutation_rate(float): likelihood of mutation (higher value means
                              that a child is more likely to mutate)
                              
    Returns:
        float: length of runtime of search in seconds
    """
    target = list(target)
    
    # characters to choose from
    characters = list(range(32, 127))
    
    # randomnly generate population
    population = []
    for _ in range(N):
        population.append(list(map(chr, list(np.random.choice(characters,
                                                              size=len(target))))))
    
    num_generations = 1
    print('TARGET:\t\t{}'.format(''.join(target)))
    output_msg = '\rOUTPUT:\t\t{}\t\tFITNESS: {:.2f}\t\tNUM GENERATIONS: {:,}'
    start = time.time()
    
    while target not in population:
        # get fitness scores for population
        results = fitness(population, target)
        
        # determine most fit population member
        leader = max(results, key=lambda x: results[x])
        print(output_msg.format(
            ''.join(leader), results[leader], num_generations), end='')
        
        # create probability list according to fitness
        prob_list = []
        for result in results:
            # append member to prob_list as many times as determined by its fitness score
            for _ in range(int(results[result]*100)):
                prob_list.append(result)
        
        # reproduce
        population = crossOver(N, characters, prob_list, mutation_rate)
        num_generations += 1
        
    end = time.time()
    print(output_msg.format(
        ''.join(target), 1.0, num_generations), end='')
    print('\nTIME:\t\t{:.2f} seconds'.format(end-start))
    return end-start

In [4]:
if __name__ == '__main__':
    num_trials = 5
    seq_records = {
        'hello': {
            'rs_time': [], 'ga_time': []
        },
        'hello world': {
            'rs_time': [],  'ga_time': []
        }
    }
    for trial in range(num_trials):
        print('Trial #{:,}\n'.format(trial+1))
        for cnt, seq in enumerate(seq_records):
            print('Random Search\n')
            rs_time = random_search(seq)
            print('\nGenetic Algorithm\n')
            ga_time = genetic_algorithm(seq)
            seq_records[seq]['rs_time'].append(rs_time)
            seq_records[seq]['ga_time'].append(ga_time)
            msg = '\nGenetic Algorithm performed ~{} times faster than Random Search.'
            print(msg.format(int(rs_time/ga_time)))
            if cnt == len(seq_records)-1 and trial == num_trials-1:
                continue
            print('='*75)
            
    print('\n\nResults after {:,} trials\n'.format(num_trials))
    for seq in seq_records:
        print('Target Sequence:', seq)
        print('\tAverage Random Search Time: {:,.2f} seconds'.format(
            np.mean(seq_records[seq]['rs_time'])))
        print('\tAverage Genetic Algorithm Time: {:,.2f} seconds\n'.format(
            np.mean(seq_records[seq]['ga_time'])))

Trial #1

Random Search

TARGET:		hello
OUTPUT:		hello
NUM ATTEMPTS:	6
TIME:		0.00 seconds

Genetic Algorithm

TARGET:		hello
OUTPUT:		hello		FITNESS: 1.00		NUM GENERATIONS: 23
TIME:		0.90 seconds

Genetic Algorithm performed ~0 times faster than Random Search.
Random Search

TARGET:		hello world
OUTPUT:		hello world
NUM ATTEMPTS:	1,070,283
TIME:		14.25 seconds

Genetic Algorithm

TARGET:		hello world
OUTPUT:		hello world		FITNESS: 1.00		NUM GENERATIONS: 202
TIME:		13.52 seconds

Genetic Algorithm performed ~1 times faster than Random Search.
Trial #2

Random Search

TARGET:		hello
OUTPUT:		hello
NUM ATTEMPTS:	97
TIME:		0.00 seconds

Genetic Algorithm

TARGET:		hello
OUTPUT:		hello		FITNESS: 1.00		NUM GENERATIONS: 16
TIME:		0.68 seconds

Genetic Algorithm performed ~0 times faster than Random Search.
Random Search

TARGET:		hello world
OUTPUT:		hello world
NUM ATTEMPTS:	69,055
TIME:		1.17 seconds

Genetic Algorithm

TARGET:		hello world
OUTPUT:		hello world		FITNESS: 1.00		NUM GENERATI